# Data Preprocessing Tools

## Importing the libraries

In [248]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Importing the dataset

In [249]:
# Load Excel files
df_s4 = pd.read_excel("0_am_workday_raw_data/resources_s4.xlsx")
df_workday = pd.read_excel("0_am_workday_raw_data/resources_workday.xlsx")

In [250]:
print(df_s4.columns)
print(df_workday.columns)

Index(['SAP Personnel #', 'Company Code', 'Hire_Date', 'Current_Role',
       'Last_Name', 'First_Name', 'Cost Center', 'SSN\n (REQ FIELD)',
       'Birthdate', 'Street Address', 'City', 'State', 'Zip Code'],
      dtype='object')
Index(['Employee_ID', 'Employee Annual Salary$', 'TAXES$',
       'Monthly Medical contribution$', 'Monthly Dental Contribution$',
       'Monthly Vision Contribution$', 'Bonus $',
       'OVERHEAD (Laptop; Desk; Supplies)$', 'Years_Of_Service',
       'Last_Promoted', 'Department', 'Location', 'Manager', 'Gender',
       'Employee HR rate', '# of Hours per week',
       'Calculated Column - Fully Loaded Cost'],
      dtype='object')


In [251]:
# Selective renaming for df_s4
df_s4_rename = df_s4.rename(columns={
    'SAP Personnel #': 'SAP_Personnel',
    'Zip Code': 'Zip_Code',
    'Company Code':'Company_Code',
    'SSN\n (REQ FIELD)':'SSN',
    'Street Address':'Street_Address'
})

# Selective renaming for df_workday
df_workday_rename = df_workday.rename(columns={
    'Employee Annual Salary$': 'Employee_Annual_Salary',
    'Monthly Medical contribution$': 'Monthly_Medical_contribution',
    'Monthly Dental Contribution$': 'Monthly_Dental_Contribution',
    'Monthly Vision Contribution$': 'Monthly_Vision_Contribution',
    'Bonus $': 'Bonus',
    'Employee HR rate': 'Employee_HR_rate',
    '# of Hours per week': 'Hours_per_week',
    'TAXES$':'TAXES',
    'OVERHEAD (Laptop; Desk; Supplies)$':'OVERHEAD',
    'Calculated Column - Fully Loaded Cost':'Fully_Loaded_Cost'
})


In [252]:
print(df_s4_rename.columns)
print(df_workday_rename.columns)

Index(['SAP_Personnel', 'Company_Code', 'Hire_Date', 'Current_Role',
       'Last_Name', 'First_Name', 'Cost Center', 'SSN', 'Birthdate',
       'Street_Address', 'City', 'State', 'Zip_Code'],
      dtype='object')
Index(['Employee_ID', 'Employee_Annual_Salary', 'TAXES',
       'Monthly_Medical_contribution', 'Monthly_Dental_Contribution',
       'Monthly_Vision_Contribution', 'Bonus', 'OVERHEAD', 'Years_Of_Service',
       'Last_Promoted', 'Department', 'Location', 'Manager', 'Gender',
       'Employee_HR_rate', 'Hours_per_week', 'Fully_Loaded_Cost'],
      dtype='object')


In [253]:
# Your selected columns
s4_req_cols = ['SAP_Personnel', 'Current_Role', 'Birthdate', 'Zip_Code']
workday_req_cols = ['Employee_ID', 'Employee_Annual_Salary', 'Monthly_Medical_contribution', 'Monthly_Dental_Contribution', 
                    'Monthly_Vision_Contribution', 'Bonus', 'Years_Of_Service', 'Last_Promoted', 'Department',
                    'Gender', 'Employee_HR_rate', 'Hours_per_week']

# Load only required columns
df_s4 = df_s4_rename[s4_req_cols]
df_workday = df_workday_rename[workday_req_cols]


In [254]:
print(df_s4.head())
print(df_workday.head())

   SAP_Personnel               Current_Role  Birthdate  Zip_Code
0             57   Senior Software Engineer 1991-05-19     94103
1             58   Associate Data Scientist 1996-03-23     94111
2             59  Associate Product Manager 1993-09-25     10009
3             60           Business Analyst 1994-03-03     60605
4             61    Chief Operating Officer 1984-01-13      2101
   Employee_ID  Employee_Annual_Salary  Monthly_Medical_contribution  \
0           57                  130800                           130   
1           58                   74250                           105   
2           59                  125850                           130   
3           60                   84750                           105   
4           61                  300000                           230   

   Monthly_Dental_Contribution  Monthly_Vision_Contribution  Bonus  \
0                           35                           19  13865   
1                           30       

## Merging the dataset

In [255]:
# Merge on SAP Personnel # and Employee_ID
df_merged = pd.merge(df_s4, df_workday, left_on='SAP_Personnel', right_on='Employee_ID', how='inner')

# Drop both ID columns since they are just identifiers
df_merged.drop(columns=['SAP_Personnel'], errors='ignore', inplace=True)

In [256]:
print(df_merged.head())

                Current_Role  Birthdate  Zip_Code  Employee_ID  \
0   Senior Software Engineer 1991-05-19     94103           57   
1   Associate Data Scientist 1996-03-23     94111           58   
2  Associate Product Manager 1993-09-25     10009           59   
3           Business Analyst 1994-03-03     60605           60   
4    Chief Operating Officer 1984-01-13      2101           61   

   Employee_Annual_Salary  Monthly_Medical_contribution  \
0                  130800                           130   
1                   74250                           105   
2                  125850                           130   
3                   84750                           105   
4                  300000                           230   

   Monthly_Dental_Contribution  Monthly_Vision_Contribution  Bonus  \
0                           35                           19  13865   
1                           30                           17   4344   
2                           35        

In [257]:
print(df_merged.shape)

(100, 15)


## Feature Engineering

In [258]:
df_merged['Last_Promoted'] = pd.to_datetime(df_merged['Last_Promoted'])
df_merged['Birthdate'] = pd.to_datetime(df_merged['Birthdate'])

today = pd.Timestamp.today()

# Calculate years since last promotion using days / 365.25
df_merged['Years_Since_Last_Promotion'] = (today - df_merged['Last_Promoted']).dt.days / 365.25

# Calculate age as integer years
df_merged['age'] = ((today - df_merged['Birthdate']).dt.days / 365.25).astype(int)


# Renaming DataFrame for feature-engineered data
df_featured = df_merged.copy()

# Optionally drop original columns
df_featured.drop(columns=['Last_Promoted', 'Birthdate'], inplace=True)

print(df_featured[['Years_Since_Last_Promotion', 'age']].head())

   Years_Since_Last_Promotion  age
0                    2.811773   34
1                    1.574264   29
2                    1.256674   31
3                    2.308008   31
4                    5.793292   41


In [259]:
print(df_featured.head())

                Current_Role  Zip_Code  Employee_ID  Employee_Annual_Salary  \
0   Senior Software Engineer     94103           57                  130800   
1   Associate Data Scientist     94111           58                   74250   
2  Associate Product Manager     10009           59                  125850   
3           Business Analyst     60605           60                   84750   
4    Chief Operating Officer      2101           61                  300000   

   Monthly_Medical_contribution  Monthly_Dental_Contribution  \
0                           130                           35   
1                           105                           30   
2                           130                           35   
3                           105                           30   
4                           230                           55   

   Monthly_Vision_Contribution  Bonus  Years_Of_Service    Department  \
0                           19  13865               5.2   Engineeri

In [260]:
print(df_featured.columns)

Index(['Current_Role', 'Zip_Code', 'Employee_ID', 'Employee_Annual_Salary',
       'Monthly_Medical_contribution', 'Monthly_Dental_Contribution',
       'Monthly_Vision_Contribution', 'Bonus', 'Years_Of_Service',
       'Department', 'Gender', 'Employee_HR_rate', 'Hours_per_week',
       'Years_Since_Last_Promotion', 'age'],
      dtype='object')


In [261]:
print(df_featured.shape)

(100, 15)


## Create Custom Dependent Column

In [262]:
# Create a boolean Series for condition
condition = df_featured['Years_Since_Last_Promotion'] > 5

# Group by that boolean condition
grouped = df_featured.groupby(condition)


In [263]:
print(grouped.size())

Years_Since_Last_Promotion
False    89
True     11
dtype: int64


In [264]:
df_featured['left'] = (df_featured['Years_Since_Last_Promotion'] > 5).astype(int)

# Renaming DataFrame for updated-engineered data
df_updated = df_featured.copy()

In [265]:
print(df_updated[['Years_Since_Last_Promotion', 'left']].head())

   Years_Since_Last_Promotion  left
0                    2.811773     0
1                    1.574264     0
2                    1.256674     0
3                    2.308008     0
4                    5.793292     1


## Export Updated Excel

In [266]:
# Create the folder if it doesn't exist
os.makedirs("1_am_workday_updated_data", exist_ok=True)

# Save the Excel file inside the folder
df_updated.to_csv("1_am_workday_updated_data/1_am_workday_updated_data.csv", index=False)
